In [55]:
import numpy as np
import random
import itertools

In [3]:
suits = {'Hearts', 'Clubs', 'Spades', 'Diamonds'}

card_name_map = {
            'Two': 2,
            'Three': 3,
            'Four': 4,
            'Five': 5,
            'Six': 6,
            'Seven': 7,
            'Eight': 8,
            'Nine': 9,
            'Ten': 10,
            'Jack': 11, 
            'Queen': 12, 
            'King': 13, 
            'Ace': 14
            }

In [4]:
class Card:
    def __init__(self, name, suit):
        self.suit = suit
        self.name = name
        self.value = card_name_map[self.name]
    
    def __str__(self):
        return '{} of {}'.format(self.name, self.suit)

In [5]:
class Deck:
    def __init__(self):
        self.deck = []
        for suit in suits:
            for card_name in card_name_map:
                self.deck.append(Card(card_name, suit))
        self.shuffle()
                
    def __str__(self):
        return ''.join(['{} \n'.format(card) for card in self.deck])
    
    def shuffle(self):
        random.shuffle(self.deck)
    
    def draw(self):
        return self.deck.pop(0)
        

In [6]:
class Player:
    def __init__(self, name, chips):
        self.chips = chips
        self.name = name
        self.hand = []
        self.pre_bet_moves = {'check', 'bet'}
        self.post_bet_moves = {'fold', 'call', 'raise'}
        
    def __str__(self):
        return '{}{}'.format('Player: {}\nChips: {}\n'.format(self.name, self.chips), ''.join(['{} \n'.format(card) for card in self.hand]))
    
    def bet(self, amount):
        self.chips -= amount
        return amount
    
    
    

In [38]:
class Table:
    def __init__(self, deck):
        self.deck = deck
        self.cards = []
    
    
        
    

In [39]:
class Game:
    def __init__(self, players, buy_in):
        self.players = [Player('{}'.format(i), buy_in) for i in range(players)]
        self.deck = Deck()
        self.table = Table(self.deck)
        self.pot = 0
        
        
    def __str__(self):
        return ''.join(['{}\n'.format(player) for player in self.players])
        
    def deal(self):
        [[player.hand.append(card) for card in [self.deck.draw() for i in range(2)]] for player in self.players]
        
    def flop(self):
        temp_flop = []
        for i in range(3):
            card = self.deck.draw()
            self.table.cards.append(card)
            temp_flop.append(card)
        
        return ''.join(['{}\n'.format(card) for card in temp_flop])
    
    def turn(self):
        temp_turn = self.deck.draw()
        self.table.cards.append(temp_turn)
        return temp_turn
    
    def river(self):
        temp_river = self.deck.draw()
        self.table.cards.append(temp_river)
        return temp_river

In [146]:
g = Game(2, 10)
g.deal()

print(g)

Player: 0
Chips: 10
King of Spades 
Eight of Clubs 

Player: 1
Chips: 10
Three of Clubs 
Nine of Hearts 




In [147]:
for card in g.players[0].hand:
    print(card)

King of Spades
Eight of Clubs


In [148]:
print(g.flop())

Four of Spades
Three of Hearts
Jack of Diamonds



In [149]:
print(g.turn())

Seven of Diamonds


In [150]:
print(g.river())

Ace of Spades


In [118]:
for card in g.table.cards:
    print(card)

Five of Spades
Eight of Diamonds
Ten of Diamonds
Five of Clubs
Three of Spades


In [119]:
combs = getScores(g.players[1], g.table)

In [145]:
for comb in combs:
    for card in comb:
        print(card)
    result = evaluateScore(comb)
    print()
    print(result['name'])
    print(result['score'])
    print()

Five of Spades
Eight of Diamonds
Ten of Clubs
Ten of Diamonds
Ace of Diamonds

Pair
2

Five of Spades
Five of Clubs
Eight of Diamonds
Ten of Clubs
Ace of Diamonds

Pair
2

Three of Spades
Five of Spades
Eight of Diamonds
Ten of Clubs
Ace of Diamonds

High Card
1

Five of Spades
Five of Clubs
Ten of Clubs
Ten of Diamonds
Ace of Diamonds

Two Pair
3

Three of Spades
Five of Spades
Ten of Clubs
Ten of Diamonds
Ace of Diamonds

Pair
2

Three of Spades
Five of Spades
Five of Clubs
Ten of Clubs
Ace of Diamonds

Pair
2

Five of Clubs
Eight of Diamonds
Ten of Clubs
Ten of Diamonds
Ace of Diamonds

Pair
2

Three of Spades
Eight of Diamonds
Ten of Clubs
Ten of Diamonds
Ace of Diamonds

Pair
2

Three of Spades
Five of Clubs
Eight of Diamonds
Ten of Clubs
Ace of Diamonds

High Card
1

Three of Spades
Five of Clubs
Ten of Clubs
Ten of Diamonds
Ace of Diamonds

Pair
2

Five of Spades
Five of Clubs
Eight of Diamonds
Ten of Diamonds
Ace of Diamonds

Pair
2

Three of Spades
Five of Spades
Eight of Diam

In [47]:
def getWinner(players, table):
    maxi = 0
    name = ""
    for player in players:
        score = getScores(player, table)
        maxi = score if score > maxi else maxi
        name = player.getName() if maxi == score else name
    print(name)
    print(maxi)
    exit()

In [105]:
def getScores(player, table):
    maxScore = 0
    cards = player.hand[:] + table.cards[:]
    combs = []
    for comb in itertools.combinations(cards, 5):
        combs.append(list(comb))
#         handScore = evaluateScore(list(comb))
#         maxScore = handScore if handScore > maxScore else maxScore
#     return maxScore
    return combs

In [108]:
def evaluateScore(hand):
    cards = hand
    # Check for a flush
    isFlush = True
    flush = cards[0].suit
    for card in cards:
        if card.suit is not flush:
            isFlush = False

    # Check for a straight
    isStraight = True
    # Sort cards by increasing rank
    cards.sort(key=lambda card_: card_.value)
    for i in range(1, len(cards)):
        if cards[i].value - 1 != cards[i - 1].value:
            isStraight = False

    # Check for royal flush
    if isStraight and isFlush:
        isRoyal = False
        # Do royal flush check
        if isRoyal:
            return {
                "hand": cards,
                "name": "Royal Flush",
                "score": 10
            }
        else:
            return {
                "hand": cards,
                "name": "Straight Flush",
                "score": 9
            }

    # Check for four of a kind, only two ways
    if cards[0].value == cards[3].value or cards[1].value == cards[4].value:
        return {
                "hand": cards,
                "name": "Quads",
                "score": 8
        }

    # Check for full house
    if ((cards[0].value == cards[1].value and cards[2].value == cards[4].value) or
            (cards[0].value == cards[2].value and cards[3].value == cards[4].value)):
        return {
                "hand": cards,
                "name": "Full House",
                "score": 7
        }

    if isFlush:
        return {
                "hand": cards,
                "name": "Flush",
                "score": 6
        }

    if isStraight:
        return {
                "hand": cards,
                "name": "Straight",
                "score": 5
        }

    # Check for three of a kind, only three ways
    if cards[0].value == cards[2].value or (cards[1].value == cards[3].value
                                                        or cards[2].value == cards[4].value):
        return {
                "hand": cards,
                "name": "Trips",
                "score": 4
        }

    # Check for two pairs, only two ways
    if ((cards[0].value == cards[1].value and cards[2].value == cards[3].value) or
            (cards[1].value == cards[2].value and cards[3].value == cards[4].value)):
        return {
                "hand": cards,
                "name": "Two Pair",
                "score": 3
        }

    # Check for a single pair
    for i in range(1, len(cards)):
        if cards[i].value == cards[i - 1].value:
            return {
                "hand": cards,
                "name": "Pair",
                "score": 2
            }
        

    # No combination, return high card
    return {
                "hand": cards,
                "name": "High Card",
                "score": 1
            }